<a href="https://colab.research.google.com/github/yorkjong/news-digest-line/blob/main/notebooks/news_notify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install

In [1]:
# @title Download Packages
import os

fns = ['clip.py', 'op.py', 'hashtag.py']

for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
    url = f'https://raw.githubusercontent.com/yorkjong/news-digest/main/src/{fn}'
    !wget $url

fns = ['line.py', 'gdrive.py']
for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
    url = f'https://raw.githubusercontent.com/yorkjong/news-digest-line/main/src/{fn}'
    !wget $url

--2024-08-15 11:35:24--  https://raw.githubusercontent.com/yorkjong/news-digest/main/src/clip.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8936 (8.7K) [text/plain]
Saving to: ‘clip.py’

clip.py             100%[===================>]   8.73K  --.-KB/s    in 0s      

2024-08-15 11:35:25 (59.2 MB/s) - ‘clip.py’ saved [8936/8936]

--2024-08-15 11:35:25--  https://raw.githubusercontent.com/yorkjong/news-digest/main/src/op.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5171 (5.0K) [text/plain]
Saving to: ‘op.py’

op.py       

In [2]:
# @title Initialize
import clip
import line
import hashtag
from gdrive import TokenTable, Subscriptions

In [3]:
# @title Setting Environment Variables for Security Data
from google.colab import userdata

os.environ['SERVICE_ACCOUNT_INFO'] = userdata.get('NEWS_NOTIFY.SERVICE_ACCOUNT_INFO')
os.environ['FOLDER_ID'] = userdata.get('NEWS_NOTIFY.FOLDER_ID')

#import json
#data = json.loads(os.environ['SERVICE_ACCOUNT_INFO'])
#json.dumps(data, separators=(',', ':'))

### Execute Actions Step by Step

In [4]:
#@title Step 1. Pick a **period** of the news { run: "auto", display-mode: "form" }
period = "Today" #@param ["Today", "Yesterday", "Recent 2 Days", "Recent 7 Days"]

def news_today():
    from datetime import datetime
    today = datetime.today().strftime('%Y_%m_%d')
    last = clip.get_recent_journal_filenames(1)[0][:-3]
    if today != last:
        print(f'last date: {last}')
        print(f'today: {today}')
    return clip.get_latest_journal()

def news_yesterday():
    fns = clip.get_recent_journal_filenames(2)
    return clip.get_journal(fns[-2])

def news_recent2days():
    return clip.merge_recent_journals(days=2)

def news_recent7days():
    return clip.merge_recent_journals(days=7)

period2func = {
    'Today': news_today,
    'Yesterday': news_yesterday,
    'Recent 2 Days': news_recent2days,
    'Recent 7 Days': news_recent7days,
}

content = period2func[period]()

import ipywidgets as widgets
output = widgets.Output()
with output:
    print(content)

In [5]:
#@title Step 2. Pick a publication **frequency** { run: "auto", display-mode: "form" }
frequency = "Daily" #@param ["Daily", "Weekly", "Any"]


subscriptions = (
    (('Tesla & SpaceX; Vehicle'), ('GroupA', 'GroupB')),
    (('Taiwan',), ('GroupA', 'GroupC')),
    (('Crypto',), ('GroupC',)),
    (('IT', 'Science', '#AI', '#Robot'), ('GroupB',)),
)

def gdrive_load_YAML(filename):
    import yaml
    path = f"/content/drive/My Drive/news-digest/{filename}"
    if os.path.exists(path):
        with open(path, 'r') as f:
             return yaml.safe_load(f)

#subscriptions = gdrive_load_YAML(f"subscriptions_{frequency}.yml")
subscriptions = Subscriptions(f"subscriptions_{frequency}.yml")

#display(subscriptions._table)

In [14]:
#@title Step 3. Line Notify { display-mode: "form" }
mock_mode = True #@param {type:"boolean"}
show_headings = True #@param {type:"boolean"}

import time
import ipywidgets as widgets
import yaml

tok_tbl = {
    'GroupA': 'LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_A',
    'GroupB': 'LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_B',
    'GroupC': 'LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_C',
}

#tok_tbl = gdrive_load_YAML("access_tokens.yml")
tok_tbl = TokenTable('access_tokens.yml')

def create_outputs():
    topics = [str(t) for t, _ in subscriptions]
    tab = widgets.Tab()
    outputs = [widgets.Output() for name in topics]
    tab.children = outputs
    for i, t in enumerate(topics):
        tab.set_title(i, t)
    display(tab)
    return outputs

def notify():
    if mock_mode:
        outputs = create_outputs()
    for i, (topics, clients) in enumerate(subscriptions):
        headings = [topic for topic in topics if not topic.startswith('#')]
        tags = [topic for topic in topics if topic.startswith('#')]
        categories = headings
        if not categories and tags:
            categories = clip.get_categories(content)
        if tags:
            lines = clip.get_lines_of_categories(categories, content, True, True)
            lines = hashtag.get_lines_with_any_hashtags(lines, tags)
            with_headings = True if headings and show_headings else False
            lines = clip.get_lines_of_categories(categories, '\n'.join(lines), False, with_headings)
        else:
            lines = clip.get_lines_of_categories(categories, content, False, show_headings)
        if not lines:
            continue
        text = clip.markdown_to_readable('\n'.join(lines))
        message = f'\n{text}'

        if mock_mode:
            with outputs[i]:
                print(message)

        for receiver in clients:
            token = tok_tbl[receiver]
            if not mock_mode:
                line.notify_message(message, token)
            #if not mock_mode:
            #    time.sleep(1)

def notify_with_checks():
    if period in ('Today', 'Yesterday'):
        if frequency not in ('Daily', 'Any'):
            print(f'The period is "{period}"')
            print('The frequency must be "Daily" or "Any"')
            return
    elif period in ('Recent 7 Days',):
        if frequency not in ('Weekly', 'Any'):
            print(f'The period is "{period}"')
            print('The frequency must be "Weekly" or "Any"')
            return
    elif period in ('Recent 2 Days',):
        if frequency not in ('2 Daily', 'Any'):
            print(f'The period is "{period}"')
            print('The frequency must be "2 Daily" or "Any"')
            return
    notify()

notify_with_checks()

### Remove Clients

#### Code

In [7]:
import copy

tbl = TokenTable('access_tokens.yml')
sub_daily = Subscriptions("subscriptions_Daily.yml")
sub_weekly = Subscriptions("subscriptions_Weekly.yml")
#sub_any = Subscriptions(f"subscriptions_Any.yml")

backup = []
for x in (tbl, sub_daily, sub_weekly):
    backup.append(copy.deepcopy(x._table))

In [8]:
import ipywidgets as widgets

choose_rm = widgets.SelectMultiple(
    options=tbl.clients(),
    #value=[options[0]],
    rows=len(tbl.clients()),
    description='Clients:',
    disabled=False,
)

btn_invalid = widgets.Button(description="Choose Invalid")
btn_remove = widgets.Button(description="Remove")
btn_recover = widgets.Button(description="Recover")
btn_save = widgets.Button(description="Save")

output2 = widgets.Output()

btns = widgets.VBox([btn_invalid, btn_remove, btn_recover, btn_save])
hbox = widgets.HBox([choose_rm, btns])
ui = widgets.VBox([hbox, output2])

def update_output():
    with output2:
        for x in (tbl, sub_daily, sub_weekly):
            display(x._table)
            print()

def on_invalid_clicked(change):
    '''Choose clients with Invalid Access Tokens.'''
    invalid = [t for t in tbl.tokens() if line.is_invalid_token(t)]
    clients_rm = tbl.clients_from_tokens(invalid)
    choose_rm.value = clients_rm
    output2.clear_output()
    with output2:
        display(invalid)
        display(clients_rm)
        print()
    update_output()

def on_remove_clicked(change):
    for i, x in enumerate((tbl, sub_daily, sub_weekly)):
        #x.remove_clients(choose_rm.value)
        del x[choose_rm.value]
    choose_rm.options = tbl.clients()
    choose_rm.rows = len(tbl.clients())
    output2.clear_output()
    update_output()

def on_recover_clicked(change):
    for i, x in enumerate((tbl, sub_daily, sub_weekly)):
        x._table = backup[i]
        backup[i] = copy.deepcopy(x._table)
    choose_rm.options = tbl.clients()
    choose_rm.rows = len(tbl.clients())
    output2.clear_output()
    update_output()

def on_save_clicked(change):
    #for x in (tbl, sub_daily, sub_weekly, sub_any):
    for x in (tbl, sub_daily, sub_weekly):
        x.save()
    output2.clear_output()
    with output2:
        print('Done!\n')
    update_output()

btn_invalid.on_click(on_invalid_clicked)
btn_remove.on_click(on_remove_clicked)
btn_recover.on_click(on_recover_clicked)
btn_save.on_click(on_save_clicked)

update_output()

#### Show

In [9]:
#display(ui)

#### Rename

In [10]:
#tbl.clients()
#for x in (tbl, sub_daily, sub_weekly):
#    x.rename('*期 / 股 *南天門未來 頂尖對談 summit meeting', '南天門未來')
#choose_rm.options = tbl.clients()
#output2.clear_output()
#update_output()

### References
news-digest:
- [Line通知訂閱連結](https://news-digest.vercel.app/LineOauth)
- [Line Notify 訂閱 news-digest 新聞](https://news-digest.vercel.app/#/page/Line%20Notify%20訂閱%20news-digest%20新聞)

Line Notify:
- [LINE Notify已連動的服務](https://notify-bot.line.me/my/)
- [上手 LINE Notify 不求人：一行代碼都不用寫的推播通知方法](https://blog.miniasp.com/post/2020/02/17/Go-Through-LINE-Notify-Without-Any-Code)
- [筆記 - 更簡單的使用 LINE Notify](https://ithelp.ithome.com.tw/articles/10231992)
- [LINE Notify ::services](https://notify-bot.line.me/my/services/)

Google Could:
- [news-digest – IAM & Admin – gdrive – Google Cloud console](https://console.cloud.google.com/iam-admin/serviceaccounts/details/101728028730105787803;edit=true/keys?authuser=1&project=gdrive-385220&supportedpurview=project)

Vercel:
- [Using the Python Runtime with Serverless Functions](https://vercel.com/docs/concepts/functions/serverless-functions/runtimes/python)

#### Format of Config Files

subscriptions_Daily.yml:
```yaml
- [['Tesla & SpaceX; Vehicle'], [GroupA, GroupB]]
- [['Tech Titans'], [GroupA, GroupB]]
- [[Finance], [GroupA, GroupB, GroupC]]
- [[Taiwan], [GroupA, GroupB, GroupC]]
- [[Crypto], [GroupA]]
- [[IT, Science, '#AI', '#Robot'], [GroupA, GroupB, GroupC]]
```

subscriptions_Weekly.yml:
```yaml
- [[Technology], [GroupA, GroupB, GroupC]]
```

access_tokens.yml:
```yaml
GroupA: LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_A
GroupB: LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_B
GroupC: LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_C
```

### Test

In [11]:
#display(subscriptions)
#display(tok_tbl._table)
#display(output) # output from Step 1

In [12]:
sub_daily.topics('Mini group')

['Tech Industry', 'Finance', 'Crypto']